데이터 로드

implicit 라이브러리 버전 0.4.0만 쓸 것. 상위 버전에선 변수 할당 오류가 일어나며, 해결할 수 없음 

In [ ]:
!pip install implicit==0.4.0

     |████████████████████████████████| 1.1MB 2.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.0-cp36-cp36m-linux_x86_64.whl size=3390274 sha256=ad0883666c7093a7bc9c209506a58e77dc5539435b170d7050ca60d6466b5525
  Stored in directory: /root/.cache/pip/wheels/fd/d1/66/84e83191d9f626c521271023e4ef83e634e3b8c154f2c4b2da
Successfully built implicit


In [ ]:
from implicit.bpr import BayesianPersonalizedRanking as BPR
from implicit.lmf import LogisticMatrixFactorization as LMF
from scipy.sparse import csr_matrix, vstack
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

In [ ]:
tr = pd.read_json("train.json")
te = pd.read_json("val.json")

tr_songs = tr.songs.tolist()
te_songs = te.songs.tolist()
tr_tags = tr.tags.tolist()
te_tags = te.tags.tolist()
te_ids = te.id.tolist()

## songs sparse matrix

In [ ]:
tr = []
iid_to_idx = {}
tag_to_idx = {} 
idx = 0

for i, l in enumerate(tr_songs):
    view = l
    for item_id in view:
        if item_id not in iid_to_idx:
            iid_to_idx[item_id] = idx
            idx += 1
    view = [iid_to_idx[x] for x in view]
    tr.append(view)

n_items = len(iid_to_idx)

idx_to_iid = {x:y for(y,x) in iid_to_idx.items()}

te = []

idx = 0
for i, l in enumerate(te_songs):
    view = l
    ret = [] 
    for item_id in view:
        if item_id not in iid_to_idx:
            continue
        ret.append(iid_to_idx[item_id])
    te.append(ret)

In [ ]:
rows = np.repeat(0,len(tr[0]))
cols = tr[0]
vals = np.repeat(1,len(tr[0]))
tr_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # train 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(tr))): # train 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(tr[i]))
    cols = tr[i]
    vals = np.repeat(1,len(tr[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    tr_csr = vstack([tr_csr, new_csr])


rows = np.repeat(0,len(te[0]))
cols = te[0]
vals = np.repeat(1,len(te[0]))
te_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # test(validation) 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(te))): # test(validation) 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(te[i]))
    cols = te[i]
    vals = np.repeat(1,len(te[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    te_csr = vstack([te_csr, new_csr]) 

r = vstack([te_csr, tr_csr]).tocoo() # test, train 데이터를 병합하고 coo_matrix로 변환

In [ ]:
def unique_item(array1, array2): # array1에 있는 원소들 중 array2에도 있는 것들을 제거하는 함수
    array1 = list(array1)
    for x in array1:
        if x in array2:
            array1.remove(x)
    return array1

## Bayesian Personalized Ranking Model for songs

In [ ]:
model = BPR(factors = 400, regularization = 0.2)
model.fit(r.T, show_progress = True)

item_result = []

In [ ]:
for u in tqdm(range(te_csr.shape[0])):
    item_recommend = model.recommend(u, tr_csr, N = 200)
    item_recommend = [x[0] for x in item_recommend]
    item_recommend = unique_item(item_recommend, te[u])
    item_recommend = [idx_to_iid[x] for x in item_recommend][:100]
    item_result.append(item_recommend)

In [ ]:
print(type(item_result), len(item_result))

<class 'list'> 23015
<class 'list'> 23015


In [ ]:
np.array(item_result).shape

(23015, 100)

(23015, 100)

## tags도 똑같이 해본다.


In [ ]:
tr = []
iid_to_idx = {}
tag_to_idx = {} 
idx = 0

for i, l in enumerate(tr_tags):
    view = l
    for item_id in view:
        if item_id not in iid_to_idx:
            iid_to_idx[item_id] = idx
            idx += 1
    view = [iid_to_idx[x] for x in view]
    tr.append(view)

n_items = len(iid_to_idx)

idx_to_iid = {x:y for(y,x) in iid_to_idx.items()}

te = []

idx = 0
for i, l in enumerate(te_tags):
    view = l
    ret = [] 
    for item_id in view:
        if item_id not in iid_to_idx:
            continue
        ret.append(iid_to_idx[item_id])
    te.append(ret)

In [ ]:
rows = np.repeat(0,len(tr[0]))
cols = tr[0]
vals = np.repeat(1,len(tr[0]))
tr_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # train 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(tr))): # train 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(tr[i]))
    cols = tr[i]
    vals = np.repeat(1,len(tr[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    tr_csr = vstack([tr_csr, new_csr])


rows = np.repeat(0,len(te[0]))
cols = te[0]
vals = np.repeat(1,len(te[0]))
te_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # test(validation) 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(te))): # test(validation) 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(te[i]))
    cols = te[i]
    vals = np.repeat(1,len(te[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    te_csr = vstack([te_csr, new_csr]) 

r = vstack([te_csr, tr_csr]).tocoo() # test, train 데이터를 병합하고 coo_matrix로 변환

## Bayesian Personalized Ranking Model for tag

In [ ]:
model = BPR(factors = 256, regularization = 0.01)
model.fit(r.T, show_progress = True)

tag_result = []

In [ ]:
for u in tqdm(range(te_csr.shape[0])):
    tag_recommend = model.recommend(u, tr_csr, N = 200)
    tag_recommend = [x[0] for x in tag_recommend]
    tag_recommend = unique_item(tag_recommend, te[u])
    tag_recommend = [idx_to_iid[x] for x in tag_recommend][:10]
    tag_result.append(tag_recommend)

In [ ]:
print(type(tag_result), len(tag_result))

In [ ]:
np.array(tag_result).shape

(23015, 10)

In [ ]:
returnval = []
for _id, rec, tag_rec in tqdm(zip(te_ids, item_result, tag_result)):
    returnval.append({
        "id": _id,
        "songs": rec[:100],
        "tags": tag_rec[:10]
    })

In [ ]:
import json
with open('ret.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(returnval, ensure_ascii=False))

## Logistic Matrix Factorization model for songs

In [ ]:

from implicit.lmf import LogisticMatrixFactorization as LMF
model_logit = LMF(factors = 50, regularization = 0.1, iterations = 40)
model_logit.fit(r.T, show_progress = True)

item_result_logit = []

for u in tqdm(range(te_csr.shape[0])):
    item_recommend = model_logit.recommend(u, tr_csr, N = 200)
    item_recommend = [x[0] for x in item_recommend]
    item_recommend = unique_item(item_recommend, te[u])
    item_recommend = [idx_to_iid[x] for x in item_recommend][:100]
    item_result_logit.append(item_recommend)


100%|██████████| 40/40 [05:24<00:00,  8.10s/it]


In [ ]:
tr = []
iid_to_idx = {}
tag_to_idx = {} 
idx = 0

for i, l in enumerate(tr_tags):
    view = l
    for item_id in view:
        if item_id not in iid_to_idx:
            iid_to_idx[item_id] = idx
            idx += 1
    view = [iid_to_idx[x] for x in view]
    tr.append(view)

n_items = len(iid_to_idx)

idx_to_iid = {x:y for(y,x) in iid_to_idx.items()}

te = []

idx = 0
for i, l in enumerate(te_tags):
    view = l
    ret = [] 
    for item_id in view:
        if item_id not in iid_to_idx:
            continue
        ret.append(iid_to_idx[item_id])
    te.append(ret)

In [ ]:
rows = np.repeat(0,len(tr[0]))
cols = tr[0]
vals = np.repeat(1,len(tr[0]))
tr_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # train 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(tr))): # train 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(tr[i]))
    cols = tr[i]
    vals = np.repeat(1,len(tr[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    tr_csr = vstack([tr_csr, new_csr])


rows = np.repeat(0,len(te[0]))
cols = te[0]
vals = np.repeat(1,len(te[0]))
te_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # test(validation) 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(te))): # test(validation) 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(te[i]))
    cols = te[i]
    vals = np.repeat(1,len(te[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    te_csr = vstack([te_csr, new_csr]) 

r = vstack([te_csr, tr_csr]).tocoo() # test, train 데이터를 병합하고 coo_matrix로 변환

## Logistic Matrix Factorization model for tags

In [ ]:
model = LMF(factors = 50, regularization = 0.1, iterations = 40)
model.fit(r.T, show_progress = True)

tag_result = []

100%|██████████| 40/40 [00:43<00:00,  1.09s/it]


In [ ]:
for u in tqdm(range(te_csr.shape[0])):
    tag_recommend = model.recommend(u, tr_csr, N = 200)
    tag_recommend = [x[0] for x in tag_recommend]
    tag_recommend = unique_item(tag_recommend, te[u])
    tag_recommend = [idx_to_iid[x] for x in tag_recommend][:10]
    tag_result.append(tag_recommend)

In [ ]:
returnval = []
for _id, rec, tag_rec in tqdm(zip(te_ids, item_result_logit, tag_result)):
    returnval.append({
        "id": _id,
        "songs": rec[:100],
        "tags": tag_rec[:10]
    })

In [ ]:
import json
with open('ret.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(returnval, ensure_ascii=False))

In [ ]:
returnval

[{'id': 118598,
  'songs': [187531,
   623047,
   207912,
   249378,
   422438,
   438857,
   421833,
   689661,
   519391,
   698536,
   233708,
   476201,
   447762,
   213438,
   133484,
   557928,
   592526,
   205939,
   232606,
   693988,
   543836,
   690767,
   570479,
   703323,
   205179,
   282501,
   205247,
   175073,
   265832,
   569715,
   307882,
   15341,
   673758,
   554509,
   439161,
   238120,
   78983,
   217622,
   161304,
   209622,
   448547,
   606374,
   640657,
   6546,
   545553,
   352807,
   483668,
   413459,
   307938,
   157900,
   164168,
   167197,
   37965,
   412246,
   484367,
   665438,
   30706,
   392638,
   551157,
   671973,
   208135,
   122737,
   622923,
   220100,
   322215,
   305452,
   331696,
   27469,
   436333,
   10690,
   523875,
   44030,
   638740,
   333100,
   545816,
   457519,
   661016,
   205312,
   650957,
   284340,
   309237,
   259770,
   73295,
   465602,
   400998,
   148997,
   443902,
   404536,
   467269,
   367